### Spectrogram based Transfer Learning

#### Data cleanup

In [1]:
import matplotlib.pyplot as plt
import os
import subprocess

from model_spectrogram import *
from utils import *

Using TensorFlow backend.


In [7]:
window_size = 12
audio_len = 24
audio_len_nn = 84
data_dir = os.path.join('..', 'audio-train-transfer')
n_samples = 112
spect_shape = (540, 960, 3)

In [3]:
cleanup_split(data_dir)
run_preprocess_spect(data_dir, str(audio_len), str(window_size), transfer = True)

#### Load pretrained model weights

In [8]:
# Build the CNN
print("Building the model..")
model = build_spectrogram_model(spect_shape, n_samples)

model.load_weights(os.path.join('..', 'neural-net-weights', \
                                        'NN_Weights_540_12', 'spect_model_weights_' + str(3) + '.h5'))

Building the model..


In [11]:
from keras.models import Model

transfer_model = Model(inputs = model.input, outputs = model.get_layer('flatten_3').output)

#### Load spectrograms as matrices

In [39]:
X, y = load_features(data_dir, str(window_size))

In [40]:
import numpy as np

X_SVM = []
for sample in range(len(X)):
    x_exp = np.expand_dims(X[sample], axis = 0)
    transfer_features = transfer_model.predict(x_exp)
    X_SVM.append(transfer_features)

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_SVM, y, test_size=0.25, random_state=42)

#### One-hot encode the inputs

In [42]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

l_enc = LabelEncoder()
l_enc.fit(y_train)
y_train_enc = l_enc.transform(y_train)
y_train_norm = np_utils.to_categorical(y_train_enc)

l_enc.fit(y_test)
y_test_enc = l_enc.transform(y_test)
y_test_norm = np_utils.to_categorical(y_test_enc)

y_train_norm = [np.where(r == 1)[0][0] for r in y_train_norm]
y_test_norm = [np.where(r == 1)[0][0] for r in y_test_norm]

#### Remove unnecessary variables from memory (To clear memory)

In [43]:
# del X, y, X_SVM, y_train_enc, y_test_enc, y_train, y_test

In [72]:
from sklearn import svm

model_SVM = svm.SVC(kernel='rbf', class_weight='balanced', C = 1, gamma = 0.0001)
# model_SVM = svm.SVC(kernel='linear', class_weight='balanced')
X_train_SVM = np.array(X_train).reshape(len(X_train), -1)

In [62]:
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import GridSearchCV

# C_range = np.logspace(-2, 10, 13)
# gamma_range = np.logspace(-9, 3, 13)
# param_grid = dict(gamma=gamma_range, C=C_range)
# cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=64)
# grid = GridSearchCV(svm.SVC(kernel='rbf', class_weight='balanced'), param_grid=param_grid, cv=cv)
# grid.fit(X_train_SVM, y_train_norm)

In [63]:
# print("The best parameters are %s with a score of %0.2f"
#       % (grid.best_params_, grid.best_score_))

In [73]:
model_SVM.fit(X_train_SVM, y_train_norm)

SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [74]:
from sklearn.metrics import accuracy_score

X_test = np.array(X_test).reshape(len(X_test), -1)
accuracy_score(y_test_norm, model_SVM.predict(X_test))

0.65384615384615385